### 기사 크롤링

- 기사 크롤링은 일반적인 크롤링과는 다르게 크롤러가 두 개 필요합니다.
- 기사 주소를 받아올 크롤러 하나와 기사 주소를 얻어왔을때 얻어온 주소를 순회하는 크롤러 하나가 필요합니다.
- 따라서 먼저 기사 주소를 가져오는 크롤러를 만들어보겠습니다.
- 링크주소는 통상적으로 a태그 내부의 href=""속성에 들어있습니다.
- 이 주소를 가져오려면 .text를 사용하지 말아야 합니다.

In [1]:
#라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import codecs

### 링크주소 크롤러 제작

- 날짜와 마지막 페이지를 입력하면 해당 날짜에 업로드된 야구 기사 주소를 긁어옵니다.
- 주소의 경우 모든 경로가 다 적혀있지 않기 때문에 이 점을 감안해서 저장해야 합니다.
- 네이버 기사의 경우 주소를 긁어온 링크대로 사용하면 이동이 안 됩니다.
    - 1. 주소 앞에 https://sports.news.naver.com 를 추가로 붙여줘야 하고
    - 2. 링크 주소 내의 ; 를 & 로 교체해줘야 합니다.

In [36]:
# 드라이버 가동
driver = webdriver.Chrome("chromedriver.exe")

# 따온 링크들을 저장할 수 있는 빈 리스트 생성
link_list = []

# base 주소
base = "https://sports.news.naver.com"
for i in range(1, 5):
    driver.get("https://sports.news.naver.com/kbaseball/news/index.nhn?date=20200118&page=%d" % i)

    # 2초 딜레이
    time.sleep(2)

    #전체 페이지 소스 가져오기

    source = driver.page_source #v

    # 파싱하기

    s1 = BeautifulSoup(source, "html.parser") #v


    # 파싱한 데이터에서 기사 영역만 가져오기

    contents = s1.find_all("div", class_="content_area")

    # 기사 링크를 획득하기 위한 a태그만 가져오기

    titles = contents[0].find_all("a", class_="title")

    # 반복문을 이용해 링크 주소만 link_list 변수에 집어 넣기

    for title in range(len(titles)):
        link_list.append(base+(str(titles[title]).split('" onclick')[0].split('href="')[1].replace(";","&")))
    print(link_list)



['https://sports.news.naver.com/kbaseball/news/read.nhn?oid=108&amp&aid=0002837234', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=472&amp&aid=0000018938', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=449&amp&aid=0000184978', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=214&amp&aid=0001008419', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=015&amp&aid=0004275865', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=119&amp&aid=0002377570', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=082&amp&aid=0000974840', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=109&amp&aid=0004152401', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=109&amp&aid=0004152393', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=529&amp&aid=0000040102', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=477&amp&aid=0000228430', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=396&amp&aid=0000

['https://sports.news.naver.com/kbaseball/news/read.nhn?oid=108&amp&aid=0002837234', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=472&amp&aid=0000018938', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=449&amp&aid=0000184978', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=214&amp&aid=0001008419', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=015&amp&aid=0004275865', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=119&amp&aid=0002377570', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=082&amp&aid=0000974840', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=109&amp&aid=0004152401', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=109&amp&aid=0004152393', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=529&amp&aid=0000040102', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=477&amp&aid=0000228430', 'https://sports.news.naver.com/kbaseball/news/read.nhn?oid=396&amp&aid=0000

In [9]:
#len(s1.find_all("a", class_="title")) #44

contents = s1.find_all("div", class_="content_area")

In [27]:
#content.find_all() #list라서

titles = contents[0].find_all("a", class_="title") #20
type(titles)

bs4.element.ResultSet

In [24]:
# titles[0].text # 링크가 없어짐
# 링크 주소 
str(titles[0]).split('" onclick')[0].split('href="')[1].replace(";","&") #문자열


'/kbaseball/news/read.nhn?oid=108&amp&aid=0002837234'

### 기사 수집 크롤러

- link_list에 저장된 주소를 이용해서 각 기사별 페이지로 이동
- 이동한 페이지에 있는 기사 본문을 txt파일에 저장

In [46]:
# 본문 순회 크롤러
driver = webdriver.Chrome("chromedriver.exe")

for url in link_list:
    driver.get(url)
    time.sleep(1)
    
    # 파싱 하기
    source1 = driver.page_source
    
    s2 = BeautifulSoup(source1,"html.parser")
    
    # 본문 기사 가져오기
    
    body = s2.find_all("div", class_="news_end font1 size3", id="newsEndContents")
    
    print(body[0].text)
    print("------------------------------------------------------------------")
        
#print(body)



						
						
							[스타뉴스 이원희 기자] 오선진. /사진=OSEN"어디서 뛰든 상관없다."한화 이글스의 내야수 오선진(31)이 팀을 위한 굳은 각오를 전했다.멀티 내야수 오선진은 지난 해 무려 122경기에 출전했다. 100경기 넘게 소화한 것은 지난 2012년(110경기) 이후 7년만. 주전 유격수 하주석(26)이 지난 해 3월 왼쪽 무릎 십자인대 파열 중상을 입은 뒤 기회를 잡았다. 갑작스러운 상황에도 오선진은 뛰어난 수비와 함께 4월말까지 타율 3할대를 유지하며 기대에 부응했다. 이후 타격 페이스가 주춤하기는 했지만, 감격스러운 주전 기회, 또 지난 해 5월 9일 SK 와이번스전에서 연타석 홈런을 때려내는 등 좋은 추억을 많이 쌓았다.오선진은 최근 스타뉴스와 통화에서 "지난 해 경기를 많이 나가서 좋았다. 하지만 체력이 떨어졌을 때 빨리 회복했어야 했는데 그러지 못했다. 준비를 잘하지 못한 것 같다. 좋기도 했지만, 한편으로는 아쉬운 시즌이었다"며 "시즌 초반에는 힘과 체력이 있어 즐겁게 경기를 했다. 하지만 몸이 힘들어지면서 반응도 느려졌다. 잘하고 싶었는데 생각대로 되지 않아 아쉬웠다"고 되돌아봤다.오선진에게 강한 목표가 생겼다. 먼저 탄탄한 체력을 만드는 것이 1차 과제다. 오선진은 "지난 시즌 덕분에 동기부여를 얻었다. 제가 지구력이 약한 것 같은데 순간 스피드, 체력 훈련 등을 한다면 좋아질 것이다. 최근 기술보다는 몸 쓰는 훈련을 많이 하고 있다. 단순히 무거운 것을 드는 것이 아니라 순간 힘을 팍팍 주면서 몸을 쓰고 있다"고 설명했다.오선진. /사진=OSEN지난 시즌 최대 수확은 유격수 포지션을 소화했다는 점이다. 오선진은 "이전에는 주로 2루와 3루 수비를 맡았다. 지난 해에는 유격수도 할 수 있다는 모습을 보여준 시즌이었다. (유격수 활약으로) 앞으로 야구를 하면서 출전 기회가 늘어날 것 같다"고 기대했다. 하주석의 복귀로 내야 경쟁이 치열해졌지만, "(하)주석이가 돌아와야 팀이 좋아질 수 있다. 저는 내야 전체를 보


 -트레이드로 한화 이글스 합류한 투수 장시환 -태안초, 태안중, 북일고 출신…프로 데뷔 14년 만에 연고지 팀 합류 -등 번호는 28번, 유니폼 받고 공식 사진 촬영까지 -“선발 자리 확정됐다 생각안 해…후배들과 경쟁 통해 발전하고 싶다”  장시환 선수, 주문하신 유니폼 나왔습니다(사진=엠스플뉴스 배지헌 기자)  [엠스플뉴스=대전]  “이제 진짜 한화 선수처럼 보이네요.” 한화 이글스 관계자의 말 그대로였다. 주황색 한화 얼트 유니폼이 멋지게 잘 어울렸다. 마치 태어날 때부터 몸에 걸치고 나온 것처럼 자연스럽게 한화 유니폼과 하나가 된 장시환은 “이제는 팀을 옮기면 안 된다. 끝까지 여기서 뛰어야 한다”며 싱긋 웃었다.  장시환은 지난해 11월 21일 포수 김현우와 함께 롯데에서 한화로 이적했다. 장시환을 얻기 위해 한화는 포수 유망주 지성준과 내야수 기대주 김주현을 롯데에 보냈다. 이 트레이드로 장시환은 2007년 프로 입단 이후 14년 만에 처음으로 연고지 팀 한화에서 뛰게 됐다. 기자가 대전 한화생명 이글스파크를 찾은 1월 18일, 장시환은 아침 일찍 부산에서 출발해 정오께 대전에 닿았다.  “부동산 계약을 하고 왔어요.” 장시환은 홈구장에서 멀지 않은 곳에 집을 얻었다. “대전에서 사는 건 이번이 처음입니다. 태안초, 태안중, 북일고까지 한화 연고지에서 자라긴 했지만 대전에서 살아본 적은 없었어요.” 현대 유니콘스에서 데뷔해 히어로즈와 KT 위즈, 롯데 자이언츠를 거쳐 이제는 한화까지. 프로 입단 뒤 벌써 세 번째로 팀을 옮기게 된 장시환이다. “매번 설레는 마음입니다.” 장시환이 말했다. “선수 생활에서 다양한 팀을 경험해본 게 장점이 될 수 있다고 봐요. 물론 한화에 왔으니 앞으론 팀을 옮기는 일은 없어야겠지만요.”  “선발 자리 확정 아냐…후배들과 경쟁해서 이겨야죠”  공식 사진을 촬영하는 장시환(사진=엠스플뉴스 배지헌 기자)  새 보금자리를 구한 장시환은 곧이어 한화생명 이글스파크 옆 한밭체육관을 찾았다. 체육관에 다다르니 분주하게 움직이는 


김태군(왼쪽)이 2017년 경남 창원시 마산야구장에서 열린 프로야구 준플레이오프 3차전 NC다이노스와 롯데 자이언츠의 경기에서 상대 타자를 태그아웃 하고 있다. 한국일보 자료사진 2019시즌 후 프로야구 자유계약선수(FA) 자격을 얻은 포수 김태군(31)이 원소속팀 NC 다이노스에 잔류한다.NC는 18일 김태군과 4년 최대 13억원에 계약했다고 밝혔다. NC에 따르면 김태군은 계약금 1억원, 연봉 2억원에 옵션 총액이 4억원이다. 김종문 NC 단장은 “김태군의 안정적인 경기 운영 능력은 다이노스의 전력 강화에 필요하다”며 “팀과 선수가 여러 방안을 함께 고민해 왔고 충분히 서로의 생각을 나눈 협상이었다”고 했다. NC 창단 멤버인 김태군은 2013년부터 2017년까지 NC의 주전 포수로 뛰다가 2017년 시즌 후 경찰청 야구단에 입대해 군 복무를 했다. 이후 NC가 2019년 시즌을 앞두고 FA로 대형 포수 양의지를 영입하면서 김태군은 주전 자리에서는 밀렸지만, 군 복귀 후 백업 포수 역할을 수행했다. 김태군은 “창단부터 함께한 다이노스에서 다시 기회를 줘서 감사하다”며 “보다 값진 선수로 팀에 공헌할 수 있도록 최선을 다하겠다”고 소감을 밝혔다. 김태군은 29일 출국하는 NC의 전지훈련 선수단에 합류해 미국 애리조나주에서 시즌을 준비한다.성환희 기자 hhsung@hankookilbo.com ▶네이버에서 한국일보 뉴스를 받아보세요 ▶뉴'잼을 보면 뉴스가 재밌다 [뉴스레터 구독하기] ▶한국일보닷컴 바로가기
기사제공 한국일보

------------------------------------------------------------------

[사진 출처 = 연합뉴스]자유계약선수(FA) 자격을 얻은 프로야구 선수 김태군(31)이 NC다이노스에 잔류한다.NC 측은 김태군과 4년 최대 13억원(계약금 1억원, 연봉 2억원, 총 옵션 4억원)에 FA 계약을 체결했다고 18일 밝혔다.김종문 NC 단장은 "김태군의 안정적인 경기 운영 능력은 다이노스의 전력 강화에 필


NC 다이노스 김태군[STN스포츠=박승환 기자]NC 다이노스가 김태군과 FA 계약을 맺었다.NC 다이노스는 "김태군과 4년 최대 13억원(계약금 1억원, 연봉 2억원, 총 옵션 4억원)에 자유계약선수(FA) 계약을 18일 체결했다"고 밝혔다.NC 김종문 단장은 "김태군의 안정적인 경기 운영 능력은 다이노스의 전력 강화에 필요하다. 팀과 선수가 여러 방안을 함께 고민해 왔고 충분히 서로의 생각을 나눈 협상이었다. 잘 기다려준 김태군에게 고마움을 전한다"고 말했다.김태군은 "창단부터 함께한 다이노스에서 다시 기회를 줘서 감사하다. 보다 값진 선수로 팀에 공헌할 수 있도록 최선을 다하겠다"고 말했다.김태군은 29일 미국 애리조나로 출발하는 다이노스 전지훈련 선수단에 합류해 시즌을 준비한다.사진=뉴시스absolute@stnsports.co.kr▶[공식 인스타그램] [공식 페이스북]▶[K팝 아이돌 연예 뉴스 보기]▶[유럽 축구 4대 리그 뉴스 보기]<Copyright ⓒ STN SPORTS. 무단전재 및 재배포 금지>
						
						
						
						
						기사제공 STN 스포츠

------------------------------------------------------------------

[KBO리그] 군입대 미룬 KIA 최원준, 주전 3루수 도전?[오마이뉴스 케이비리포트 기자]2020시즌 신임 윌리엄스 감독과 함께 반등을 노리는 KIA 타이거즈에 비상등이 켜졌다. FA 자격을 취득한 안치홍이 지난 6일 롯데 자이언츠로 이적했다. 김주찬은 연말 지방종 제거 수술을 받아 전지훈련에 참가할 수 없다. KIA는 지난해 야수 주전 2명이 빠진 상태로 2020시즌 개막을 맞이하게 된다. 그렇지 않아도 베테랑의 비중이 높았던 KIA 야수진의 세대교체는 가속화될 전망이다.              ▲  2019년 커리어 로우를 기록한 KIA 최원준            ⓒ KIA 타이거즈   KIA 야수진의 기대주 중 한 명은 최원준이다. 2016년


NC 다이노스 김태군. 스포츠조선DB.[스포츠조선 선수민 기자] NC 다이노스가 포수 김태군과 FA 계약을 체결했다.NC는 18일 "김태군과 4년 총액 13억원(계약금 1억원, 연봉 2억원, 옵션 총 4억원)에 계약을 맺었다"고 발표했다. 생애 첫 FA 자격을 획득한 김태군은 긴 협상 끝에 NC에 잔류했다. 이로써 NC는 FA 선수들과의 협상을 마무리했다.지난 8월 경찰 야구단에서 제대한 김태군은 FA를 앞두고 있어 뜨거운 관심을 받았다. 포수가 취약한 롯데 자이언츠가 시즌 중 트레이드를 시도하기도 했다. 포수 품귀 현상으로 FA 시장에서 가치가 치솟을 것으로 예상됐다. 그러나 막상 열린 FA 시장에는 찬바람이 불었다. 롯데가 일찌감치 트레이드로 포수 지성준을 영입하면서 상황이 달라졌다. 수요가 없어 몸값도 자연스럽게 낮아졌다.원 소속팀 NC도 급할 게 없었다. 보상 선수 출혈을 감수하면서까지 김태군을 원하는 구단은 나타나지 않았다. NC는 이미 지난해 통 큰 투자로 양의지를 영입했고, 올해 포수 유망주 김형준이 1군에서 성장 가능성을 보였다. 반면 김태군은 시즌 막판 합류해 18경기에서 타율 1할8푼2리에 그쳤다. 활용도가 떨어지면서 협상도 길어졌다. 다만 NC는 "당연한 협상의 과정이라 생각한다. 김태군과 계약하면 엔트리 활용이 수월해진다. 포수는 다양성이 필요하다. 좋은 부분이 있다"고 설명했다.결국 줄다리기 끝에 김태군은 NC와 계약했다. 'FA 대박'에는 실패했지만, 캠프 출발 전 협상을 완료했다.NC 김종문 단장은 계약 직후 "김태군 선수의 안정적인 경기 운영 능력은 다이노스의 전력 강화에 필요하다. 팀과 선수가 여러 방안을 함께 고민해 왔고 충분히 서로의 생각을 나눈 협상이었다. 잘 기다려준 태군 선수에게 고마움을 전한다"고 말했다.김태군도 "창단부터 함께한 다이노스에서 다시 기회를 줘서 감사하다. 보다 값진 선수로 팀에 공헌할 수 있도록 최선을 다하겠다"고 말했다. 선수민 기자 sunsoo@sportschosun.com▶2020 신년운세 보러가


[OSEN=광주, 이선호 기자] 배번 8번의 명성을 누가 이을까? FA 안치홍(31)이 롯데 자이언츠로 이적하면서 배번의 행방이 관심이다. 안치홍은 2009년 고졸 루키로 입단하면서 '8번'을 달았다. 이후 작년까지 11년 동안 '8번'의 주인공은 바뀌지 않았다. '3번'을 달았던 키스톤콤비 김선빈과 '38' 콤비로 10년 동안 그라운드를 누벼 팬들에게도 익숙하다. 두 선수가 2014시즌을 마치고 동시에 군입대하면서 배번이 비워졌다. 그러나 당시 김기태 신임감독은 "두 선수는 KIA의 간판스타이다. 돌아올 때까지 3번과 8번은 그대로 공번으로 놔두겠다"며 임시결번 처리를 했다. 기둥 선수들에 대한 예우를 확실히했다. 말을 전해들은 두 선수는 고마움을 표시했고 복귀하자마자 2017 통합 우승으로 보답했다. 안치홍은 롯데에 입단하면서 배번을 8번이 아닌 13번으로 선택했다. 배번 8번은 외야수 전준우가 달고 있어 선택이 불가능했다. 13번은 서울고 시절과 경찰청 시절 달았던 번호였다. 프로의 꿈을 꾸었던 고교시절의 초심으로 돌아가 야구를 하겠다는 의지로 읽힌다. KIA에서 '8번'은 당분간 공번이 될 것으로 보인다. 안치홍의 보상선수로 영입한 투수 김현수는 '50번'을 택했다. 김현수는 롯데 시절 43번을 달았다. 한 자리 배번은 주로 야수들의 번호인 점도 고려해 '50번'을 낙점한 것으로 보인다. 더욱이 시기적으로 8번을 달 수 있는 선수들이 없었다. 구단은 연말에 외국인 선수, 나주환, 홍상삼, 변진수, 신인 등 새로 입단한 선수들의 배번 배정을 모두 마쳤다. 안치홍이 배번 정리가 된 직후에 팀을 떠나 '8번'이 자연스럽게 공석이 되었다. 결국 추후 트레이드 혹은 외국인 선수 교체 등이 있을 때나 변동이 있을 수 있다. 그러나 아무에게나 '8번'을 주지는 않을 것으로 보인다.  KIA의 영구결번은 선동렬의 18번, 이종범의 7번이다. 한국야구의 중흥기를 이끌었던 전설들의 번호이다. 안치홍은 두 전설에는 미치지 못했지만 KIA 11년 동안 성실함과 견실한 활약


KIA 타이거즈 최형우가 2017년 한국시리즈에서 우승을 확정한 직후 동료들과 기뻐하고 있다. 최형우는 올해가 4년 계약의 마지막 해로 정상적으로 시즌을 마치면 두 번째 FA 자격을 얻는다. 스포츠조선 DB[스포츠조선 노재형 기자] KIA 타이거즈 최형우는 FA 몸값 100억 시대를 연 주인공이다.KIA 구단이 2016년 11월 24일 "FA 최형우와 4년 100억원에 계약을 체결했다"고 발표했을 때 '드디어 올 것이 왔구나'라는 반응이 대부분이었다. '보장 금액만 100억이다', '옵션을 합치면 훨씬 많다', '몸값 거품이 더욱 심해질 것이다' 등 주위의 수근거림에도 최형우는 계약 첫 시즌인 2017년 KIA를 정규시즌과 한국시리즈 우승으로 이끌며 '제대로 돈값을 한 FA'라는 평가를 받았다.최형우라는 4번타자를 장착한 KIA는 김선빈 안친홍 나지완 이범호 버나디나 등 동료 타자들도 시너지 효과를 내 타선 폭발력이 배가 됐다. 중심타자 가세가 타선을 어떻게 바꿔놓은 지를 매우 잘 보여준 케이스가 2017년 KIA 타이거즈다. 최형우는 2018년 타율 3할3푼9리, 25홈런, 103타점을 올리며 활약을 이어갔지만, 팀은 와일드카드 결정전에서 탈락했다. 그리고 지난 시즌 최형우는 136경기에 출전해 타율 3할을 겨우 기록했지만, 17홈런, 86타점에 그치며 KIA 이적 후 처음으로 슬럼프를 맛봤다. 공인구 반발계수 감소의 직격탄을 최형우도 피하기 힘들었다. 전반적인 타선 침체 속에 KIA는 페넌트레이스 7위로 가을야구 무대에 나가지 못했다. 최근 3년 동안 가장 힘들었던 시즌이다. 최형우는 이제 FA 계약 마지막 시즌을 준비한다. KIA는 올해 전지훈련을 미국 플로리다주 포트마이어스에서 실시한다. KBO리그 세 번째 외국인 사령탑 맷 윌리엄스 감독 체제로 맞는 첫 전지훈련으로 KIA 최형우에게는 세 번째 전훈캠프다. KIA는 최형우를 비롯해 역대 구단 최다인 74명의 대규모 선수단을 꾸리고 오는 30일 출국한다. 최형우는 올시즌을 정상적으로 마치면 생애 두 번


사진은 해피메리드컴퍼니(유튜브 봉드)두산베어스 이영하가 결혼한다.해피메리드컴퍼니 측은 18일 "이영하가 승무원 여자친구와 오늘 오후 6시 서울 삼성동 그랜드힐에서 결혼식을 올린다"라고 밝히며 웨딩화보를 공개했다. 이날 결혼식의 사회는 개그맨 박성광이 맡으며, 축가는 이석훈이 부른다.이영하는 키움 히어로즈 이정후, KT wiz 강백호, 두산 베어스 박치국 등 절친 야구 선수들과 웨딩화보를 함께 찍었다. 이들은 각팀의 유니폼을 착용하고 신부 측 들러리들과 화보 촬영에 임해 훈훈함을 자아냈다.이영하는 1997년생으로 올해 24살이며 2016년 1차 지명으로 두산 유니폼을 입었다.박정미 부산닷컴 기자 likepea@busan.com▶ 네이버에서 부산일보 구독하기 클릭!▶ '터치 부산' 앱 출시, 부산일보 지면을 내 손 안에!▶ 부산일보 홈 바로가기
기사제공 부산일보

------------------------------------------------------------------

한국은 지난해 11월 프리미어12에서 준우승을 차지해 WBSC 세계랭킹 3위를 유지했다. 박재만 기자 pjm@sportschosun.com[스포츠조선 노재형 기자] 한국 야구가 세계 랭킹 3위를 유지했다.세계야구소프트볼연맹(WBSC)이 17일 발표한 순위에서 한국은 4622점으로 3위를 기록했다. 일본이 6127점으로 1위를 지켰고, 미국이 4676점을 얻어 2위, 한국에 이어 4352점을 기록한 대만이 4위에 올랐다. 1~4위가 지난해와 같다.WBSC는 최근 4년간의 야구 국가대표팀 성적을 바탕으로 순위를 매긴다. 한국은 지난해 11월 프리미어12에서 일본에 이어 준우승을 차지해 랭킹 포인트 1102점을 추가했다. 또 지난해 10월 아시아야구선수권대회 4위로 74점, 지난해 8월 세계청소년선수권대회 3위로 418점을 추가했다.일본은 프리미어12에서 우승을 차지하면서 랭킹 포인트 1380점을 추가했고, 앞서 아시아선수권대회 준우승으로 105점을 얻어 1위를 굳건히 했다.프리미어12에

KeyboardInterrupt: 